In [302]:
import os
import numpy as np
from scipy import fftpack as fftp
import pycwt

from matplotlib import pyplot as plt
# axes_grid1 should be deprecated soon but this method is, seemingly, still not fully ported
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [314]:
# High DPI plots take quite a bit to render, so keeping it down for the notebook speeds up things
FINAL_PROOF = True
if FINAL_PROOF:
    plt.rcParams['figure.dpi'] = 1200

def savefig(name: str, figure=None):
    if FINAL_PROOF:
        fmt = 'eps'
        fname = name + '.eps'
        dpi = 1200
        save_folder = 'figures'
    else:
        fmt = 'png'
        fname = name + '.png'
        dpi = 100
        save_folder = 'draft'
    if figure is None:
        plt.savefig(os.path.join(save_folder, fname), format=fmt, dpi=dpi)
    else:
        figure.savefig(os.path.join(save_folder, fname), format=fmt, dpi=dpi)

In [247]:
freq_1 = 10
freq_2 = 40
x_max = 5
N = 10000
step = 2 * x_max / N
x_values = np.linspace(-x_max, x_max, N)
flattop_bell_curve = gennorm.pdf(x_values, 6)
step_tophat = np.zeros_like(x_values)
step_tophat[4*N//10:6*N//10] = 0.5
y_values_cont = np.cos(freq_1 * x_values * (2 * np.pi)) + 0.1 * np.cos(freq_2 * x_values * 2 * np.pi)
y_values_part = np.cos(freq_1 * x_values * 2 * np.pi) + flattop_bell_curve * np.cos(freq_2 * x_values * 2 * np.pi)
# y_values4 = np.cos(freq_1 * x_values * 2 * np.pi) + step_tophat * np.cos(freq_2 * x_values * 2 * np.pi)

Fourier spectra (power spectral density)

In [248]:
fftfreq = fftp.fftfreq(N, d=step)
fft_cont = fftp.fft(y_values_cont)
fft_part = fftp.fft(y_values_part)
psd_cont = np.abs(fft_cont[0:]) ** 2
psd_part = np.abs(fft_part[0:]) ** 2

Wavelet calculations. Given the frequencies are on the order of tens of Hz, we need negative powers to describe periods (log2(1/40) < 0). Cones of influence will be the same for both plots.

In [249]:
freqs = 1 / (pycwt.Morlet(6.).flambda() * np.power(2, np.arange(-7, 2, 0.1)))
# We only need the power spectrum and the cone of influence
coefs_cont, _, _, coi, *_ = pycwt.cwt(y_values_cont, step, pycwt.Morlet(6.), freqs=freqs)
coefs_part, _, _, coi, *_ = pycwt.cwt(y_values_part, step, pycwt.Morlet(6.), freqs=freqs)
power_cont = (abs(coefs_cont)) ** 2
power_part = (abs(coefs_part)) ** 2
period = 1. / freqs
# Alternatively, get scales from the pycwt call and normalize by them.
power_cont /= np.sum(power_cont)
power_part /= np.sum(power_part)
contourlevels = np.arange(-35, -15)

In [313]:
# # Raw signals
fig, ax = plt.subplots(3, 2, constrained_layout=True, sharey='row', sharex='row', figsize=(12, 9))

ax_signal = ax[0]
ax_signal[0].plot(x_values, y_values_cont)
ax_signal[1].plot(x_values, y_values_part)
ax_signal[0].set_ylim(-2, 2)
ax_signal[0].set_xlim(-1, 1)
ax_signal[0].set_ylabel('Signal values, a.u.')

ax_fft = ax[1]
ax_fft[0].plot(fftfreq[fftfreq > 0], psd_cont[fftfreq > 0])
ax_fft[1].plot(fftfreq[fftfreq > 0], psd_part[fftfreq > 0])

axins_fft0 = ax_fft[0].inset_axes([0.45, 0.42, 0.47, 0.47])
axins_fft0.plot(fftfreq[fftfreq > 0], psd_cont[fftfreq > 0])
axins_fft0.set_xlim(39, 41)
axins_fft0.set_ylim(0, 3e5)
axins_fft0.set_xticks([39, 39.5, 40, 40.5, 41], [39, '', 40, '', 41])
axins_fft0.set_yticks([0, 2e5], ['0', '2e5'])
# ax_fft[0].indicate_inset_zoom(axins_fft0, edgecolor="black")
mark_inset(ax_fft[0], axins_fft0, loc1=3, loc2=1, fc='none', ec='black')
axins_fft1 = ax_fft[1].inset_axes([0.45, 0.42, 0.47, 0.47])
axins_fft1.plot(fftfreq[fftfreq > 0], psd_part[fftfreq > 0])
axins_fft1.set_xlim(39, 41)
axins_fft1.set_ylim(0, 3e5)
axins_fft1.set_xticks([39, 39.5, 40, 40.5, 41], [39, '', 40, '', 41])
axins_fft1.set_yticks([0, 2e5], ['0', '2e5'])
mark_inset(ax_fft[1], axins_fft1, loc1=3, loc2=1, fc='none', ec='black')
# ax_fft[1].indicate_inset_zoom(axins_fft1, edgecolor="black")


ax_fft[0].set_xlim(0, 42)
ax_fft[0].set_xlabel('Frequency, Hz')
ax_fft[1].set_xlabel('Frequency, Hz')
ax_fft[0].set_ylabel('Power spectral density')

ax_wavelet = ax[2]
ax_wavelet[0].contourf(x_values, np.log2(period), np.log2(power_cont), contourlevels, extend='both', cmap='coolwarm')
ax_wavelet[1].contourf(x_values, np.log2(period), np.log2(power_part), contourlevels, extend='both', cmap='coolwarm')
# COI
ax_wavelet[0].plot(x_values, np.log2(coi), '0.8', lw=4)
ax_wavelet[0].plot(x_values, np.log2(coi), 'w--', lw=3)
# Original frequencies for the reference
ax_wavelet[0].plot(x_values, [np.log2(1/40)] * len(x_values), 'k--')
ax_wavelet[0].plot(x_values, [np.log2(1/10)] * len(x_values), 'k--')
# The same for the second plot
ax_wavelet[1].plot(x_values, np.log2(coi), '0.8', lw=4)
ax_wavelet[1].plot(x_values, np.log2(coi), 'w--', lw=3)
ax_wavelet[1].plot(x_values, [np.log2(1/40)] * len(x_values), 'k--')
ax_wavelet[1].plot(x_values, [np.log2(1/10)] * len(x_values), 'k--')
# Y axis
wavelet_yticks = np.arange(-7, 2, 1)
ax_wavelet[0].set_ylim(-7, 2)
# ax_wavelet[0].set_yticklabels(1 + 1 / np.power(2.0, wavelet_yticks))
ax_wavelet[1].set_yticks(wavelet_yticks, 1 / np.power(2.0, wavelet_yticks))
ax_wavelet[0].invert_yaxis()
# subfigs[2].supxlabel('x')
ax_wavelet[0].set_ylabel('Frequency, Hz')
ax_wavelet[0].set_xlabel('x (time), s')
ax_wavelet[1].set_xlabel('x (time), s')
savefig('fft-wavelet-comparison', fig)